In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import tensorflow.keras.layers as tfl
import tensorflow.keras as keras
from tensorflow.keras.utils import image_dataset_from_directory

%matplotlib inline

In [ ]:
img1 = mpimg.imread('/kaggle/input/state-farm-distracted-driver-detection/imgs/train/c0/img_100026.jpg')
plt.imshow(img1)
img1.shape

In [ ]:
img2 = mpimg.imread('/kaggle/input/state-farm-distracted-driver-detection/imgs/train/c0/img_101196.jpg')
plt.imshow(img2)
img2.shape

In [ ]:
train_data = image_dataset_from_directory('/kaggle/input/state-farm-distracted-driver-detection/imgs/train',
                                         labels='inferred',
                                          validation_split=0.2,
                                          subset="training",
                                          seed=123,
                                         batch_size=32,
                                    image_size=(480, 640),)

val_data = image_dataset_from_directory('/kaggle/input/state-farm-distracted-driver-detection/imgs/train',
                                         labels='inferred',
                                          validation_split=0.2,
                                          subset="validation",
                                          seed=123,
                                         batch_size=32,
                                    image_size=(480, 640),)

In [ ]:
train_data.class_names

In [ ]:
model = keras.Sequential([
    tfl.Conv2D(32, 5,strides =  1, padding = 'same', activation = 'relu', input_shape=[480, 640, 3]),
    tfl.Conv2D(32, 5,strides =  1, padding = 'same', activation = 'relu'),
    tfl.MaxPool2D(),
     
    tfl.Conv2D(64, 3,strides =  1, padding = 'same', activation = 'relu'),
     tfl.Conv2D(64, 3,strides =  1, padding = 'same', activation = 'relu'),
     tfl.MaxPool2D(),
     
     tfl.Conv2D(128, 3,strides =  1, padding = 'same', activation = 'relu'),
     tfl.Conv2D(128, 3,strides =  1, padding = 'same', activation = 'relu'),
     tfl.MaxPool2D(),
     
     tfl.Flatten(),
      tfl.Dense(units=10, activation="relu"),
      tfl.Dense(units=10),
    ])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
driver_img_list = pd.read_csv('/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
driver_img_list

In [ ]:
normalization_layer = tfl.Rescaling(1./255)
train_data = train_data.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
history = model.fit(train_data,validation_data = val_data, epochs = 10)

In [ ]:
test_data = image_dataset_from_directory('/kaggle/input/state-farm-distracted-driver-detection/imgs/test',
                                     labels = None, 
                                     image_size = (480, 640))

test_data = test_data.map(lambda x: normalization_layer(x))
predictions = model.predict(test_data)
predictions

In [ ]:
y_pred = tf.nn.softmax(predictions)
y_pred

In [ ]:
sub = pd.DataFrame({'img': [f'img_{i}' for i in range(79726)],'c0': y_pred[:, 0], 'c1': y_pred[:, 1], 'c2': y_pred[:, 2], 'c3': y_pred[:, 3],
                   'c4': y_pred[:, 4], 'c5': y_pred[:, 5], 'c6': y_pred[:, 6], 'c7': y_pred[:, 7],
                   'c8': y_pred[:, 8], 'c9': y_pred[:, 9]})
sub

In [ ]:
sub.to_csv('submission.csv', index = False, header=True)